# PMR3401 - Mecânica Computacional para Mecatrônica
## Exercício-Programa 1
João Rodrigo Windisch Olenscki ($\texttt{10773224}$) </br>
Lui Damianci Ferreira ($\texttt{10770579}$)

### Equações, constantes e condições iniciais
 $$ m_{total}\ddot{x} + m_1 L (\theta^2 \cos{\theta} + \ddot{\theta}\sin{\theta}) = -F + F_1 \cos{\theta}$$
 
 $$ m_1 \ddot{x} L \sin{\theta} + m_1 \ddot{\theta} L^2 + 2 I \omega \dot{\theta} = 0 $$
 
 Constantes:
 
 $m_{total}=1939$ kg </br>
 $L=2.95$ m </br>
 $I = 1$ kg m² </br>
 $\omega=10$ rad/s </br>
 
 Forças no eixo: </br>
 1. Se tem tração no eixo: $F = \mu m g$ </br>
 2. Se não tem tração no eixo: $F=\beta m g$ </br>
 
 * $\mu=0.42$: coeficiente de atrito na roda quando há **rolamento** </br>
 * $\beta=0.02$: coeficiente de atrito da roda quando há **deslizamento**
 
 Obs.: nestas forças o termo de massa $m$ corresponde a **massa do eixo**, sendo uma fração da massa total (dependendo da distribuição inputada) </br>
 Obs².: eixos tracionados rolam e eixos não-tracionados deslizam

In [23]:
import numpy as np
import matplotlib.pyplot as plt
# usar latex no matplotlib: https://stackoverflow.com/questions/41453109/how-to-write-your-own-latex-preamble-in-matplotlib
